In [1]:
# from pyDWSIMconnect import PyDWSIMconnect
from simulation import Simulation
import numpy as np
import time
from fobj import *
from scipy import optimize
from pprint import pprint

In [2]:
# Loading DWSIM simulation into Python (Simulation object)
sim_smr = Simulation(path2sim="C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz")
sim_smr.Add_refs()
sim_smr.Connect()

added refs
Simulation was loaded successfully


In [3]:
def fobj8n_2exp(sim_smr, x):
    mr1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("MR-1")
    comp1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-1") 
    comp2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-2") 
    comp3 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-3") 
    comp4 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-4") 
    pump1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01") 
    pump2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-02") 
    vlv1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("VALV-01")
    cool8 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-08")
    lng = sim_smr.flowsheet.GetFlowsheetSimulationObject("LNG-1")
    if sim_smr.x is None:
        sim_smr.x = np.zeros(len(x))
    if np.linalg.norm(sim_smr.x - np.asarray(x))>1e-10:
    # m_old = np.ones(mr1.Phases[0].Compounds.Values.Count)
    # ite=0
    # for m_i in mr1.Phases[0].Compounds.Values:
    #     m_old[ite] = m_i.MassFraction*mr1.GetMassFlow()
    #     ite+=1
    # x_old = np.array( [m_old[7],m_old[0],m_old[1],m_old[2],m_old[5], vlv1.OutletPressure, comp4.POut, cool8.OutletTemperature] ) 
    # if np.linalg.norm( x-x_old ) > 1e-5:
    # if ( abs(mr1.GetMassFlow() - x[0])/abs(x[0]) + abs(vlv1.OutletPressure - x[1])/abs(x[1]) + abs(comp4.POut - x[2])/abs(x[2]) ) > 1e-12:
        # print("calculated with ")
        # print( abs(mr1.GetMassFlow() - x[0])/abs(x[0]))
        # print(abs(vlv1.OutletPressure - x[1])/abs(x[1]))
        # print(abs(comp4.POut - x[2])/abs(x[2]) )
        # mr1.SetMassFlow(x[0])
        sep1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-02")
        sep2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-03")
        sep1.GraphicObject.Activate = False
        sep2.GraphicObject.Activate = False

        mr1.SetOverallCompoundMassFlow(0,x[1])
        mr1.SetOverallCompoundMassFlow(1,x[2])
        mr1.SetOverallCompoundMassFlow(2,x[3])
        mr1.SetOverallCompoundMassFlow(5,x[4])
        mr1.SetOverallCompoundMassFlow(7,x[0])
        vlv1.OutletPressure = x[5]
        comp4.POut = x[6]
        cool8.OutletTemperature = x[7]

        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction < 1e-5:
            pump1.GraphicObject.Active = False
            # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01").GraphicObject.Active)
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump1.GraphicObject.Active = True
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject) #avoid bug
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,
                -1,-1)
        sep1.GraphicObject.Active = True
        sep1.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction < 1e-5:
            pump2.GraphicObject.Active = False
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump1.GraphicObject.Active = True
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,
                -1,-1)
        sep2.GraphicObject.Active = True
        sep2.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        #force to calculate spec2 spec3 cool6 e cool5
        sim_smr.flowsheet.GetFlowsheetSimulationObject("SPEC-02").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("SPEC-03").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-06").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-05").Solve()
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        sumW = (comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction > 1e-5:
            sumW += pump1.DeltaQ 
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction > 1e-5:    
            sumW += pump2.DeltaQ
        sumW = sumW/lng.GetMassFlow()/3600
        mita1 = min(sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-08").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-09").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(), 
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-1_2").GetTemperature()   - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-10").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-11").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-2").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature()
        )
        mita2 = min(sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-10").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-16").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-2").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-16").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-12").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-13").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-3").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-13").GetTemperature()
        )
    else:
        sumW = sim_smr.f
        mita1 = sim_smr.g
        mita2 = sim_smr.g
        # print("skipped with")
        # print( abs(mr1.GetMassFlow() - x[0])/abs(x[0]))
        # print(abs(vlv1.OutletPressure - x[1])/abs(x[1]))
        # print(abs(comp4.POut - x[2])/abs(x[2]) )
    # if np.linalg.norm( x-x_old ) > 1e-5:
    #     sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.path,True)
    sim_smr.x = x
    sim_smr.f = sumW
    sim_smr.g = min(mita1, mita2)
    # print(sim_smr.x,
    #     sim_smr.f,
    #     sim_smr.g)
    return sumW, min(mita1, mita2)

In [5]:
# Initial simulation setup
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -40+273.15] )
sim_smr.fobj = lambda x: fobj8n_2exp(sim_smr, x)
sumW, mita = sim_smr.fobj(0.95*x0)
print(sim_smr.x,
sim_smr.f,
sim_smr.g)
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,
        'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep2.dwxmz',True)

[6.59722222e-05 1.84722222e-04 2.63888889e-04 2.90277778e-04
 4.75000000e-04 2.37500000e+05 4.75000000e+06 2.21492500e+02] 0.37954699972061173 4.745016361192398
[6.59722222e-05 1.84722222e-04 2.63888889e-04 2.90277778e-04
 4.75000000e-04 2.37500000e+05 4.75000000e+06 2.21492500e+02] 0.37954699972061173 4.745016361192398


In [5]:
print(sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01").GraphicObject.Active)

True


In [15]:
x_top = np.array( [0.252779/3600, 0.441063/3600, 1.411068/3600, 0.805952/3600, 1.851257/3600, 3.342832e5, 34.97221e5, -27.2958+273.15] )
args=(sim_smr)
xtol=0.01
ftol=0.01
maxiter=50 # +- 5 seconds per iteration
print("starting optimization")
iteration = 0
regularizer = np.array([1e4,1e4,1e4,1e4,1e4, 1e-5, 1e-6, 1e-2])
#rever \/
bounds_raw = np.array( [0.6*np.asarray(x0), 1.4*np.asarray(x0)] )
bounds_raw[0][-1] = 153
bounds_raw[1][-1] = 233
bounds_reg = regularizer*bounds_raw
print(bounds_reg)
bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])
g = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[1]
nonlinear_constraint = optimize.NonlinearConstraint(g, 3, np.inf, jac='2-point', hess=optimize.BFGS())
f = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[0]
# print(fobj8n_2exp(sim_smr,x_top))

starting optimization
[[0.41666667 1.16666667 1.66666667 1.83333333 3.         1.5
  3.         1.53      ]
 [0.97222222 2.72222222 3.88888889 4.27777778 7.         3.5
  7.         2.33      ]]


In [8]:
fobj8n_2exp(sim_smr, x_top)
pprint(vars(sim_smr))

[7.02163889e-05 1.22517500e-04 3.91963333e-04 2.23875556e-04
 5.14238056e-04 3.34283200e+05 3.49722100e+06 2.45854200e+02] 0.2878099850939581 2.1030562816582687
{'f': 0.2878099850939581,
 'flowsheet': <DWSIM.UI.Desktop.Shared.Flowsheet object at 0x000002385FFB6DC0>,
 'fobj': <function <lambda> at 0x000002385FF69820>,
 'g': 2.1030562816582687,
 'interface': <DWSIM.Automation.Automation2 object at 0x000002385FFB32E0>,
 'path': 'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz',
 'path2dwsim': 'C:\\Users\\lfsfr\\AppData\\Local\\DWSIM7\\',
 'x': array([7.02163889e-05, 1.22517500e-04, 3.91963333e-04, 2.23875556e-04,
       5.14238056e-04, 3.34283200e+05, 3.49722100e+06, 2.45854200e+02])}


In [9]:
# Local optimization with trust-region -> working to some extent
result = optimize.minimize( f, np.asarray(x0)*regularizer, 
            method='trust-constr', jac='2-point', hess=optimize.BFGS(),
            constraints=[nonlinear_constraint], bounds=bounds, callback=None,
            options={'verbose': 3, 
                     'xtol': xtol, 
                     'maxiter': 0.1*maxiter, 
                     'finite_diff_rel_step': None, 
                     'initial_tr_radius': 0.1} )

[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444459e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444444e-05 1.94444447e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444444e-05 1.94444444e-04 2.77777782e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555560e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000007e-04 2.50000000e+05 5.00000000e+06 2.33150000e+02] 0.39330115399949256 6.617040555794176
[6.94444444e-05 1.94444444e-04 2.7

c:\Users\lfsfr\Desktop\pyDWSIMopt\venv\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444459e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444444e-05 1.94444447e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444444e-05 1.94444444e-04 2.77777782e-04 3.05555556e-04
 5.00000000e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555560e-04
 5.00000000e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000007e-04 2.59297746e+05 4.96404362e+06 2.51093263e+02] 0.38656753490716256 -0.18041919814538687
[6.94444444e-05 

In [19]:
print(f(result.x))
print(g(result.x))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,
        'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep2.dwxmz',True)
print(result.x)
# x_top
bounds


[5.07711576e-05 1.58815526e-04 2.40359276e-04 2.62695931e-04
 5.83489143e-04 2.29465431e+05 3.34589687e+06 1.94859203e+02] 0.31681716003415783 3.7783261196048556
0.31681716003415783
[5.07711576e-05 1.58815526e-04 2.40359276e-04 2.62695931e-04
 5.83489143e-04 2.29465431e+05 3.34589687e+06 1.94859203e+02] 0.31681716003415783 3.7783261196048556
3.7783261196048556
[0.50771158 1.58815526 2.40359276 2.62695931 5.83489143 2.29465431
 3.34589687 1.94859203]


Bounds(array([0.41666667, 1.16666667, 1.66666667, 1.83333333, 3.        ,
       1.5       , 3.        , 1.53      ]), array([0.97222222, 2.72222222, 3.88888889, 4.27777778, 7.        ,
       3.5       , 7.        , 2.33      ]))

In [17]:
# Global optimization with Differential Evolution
result = optimize.differential_evolution(f, bounds, constraints=[nonlinear_constraint], 
            maxiter=5, popsize=1, seed=1234, disp=True, polish=False)

[7.06872180e-05 1.40333006e-04 2.44922670e-04 3.96101478e-04
 3.12999597e-04 2.34581948e+05 6.94014674e+06 1.91341865e+02] 0.37941377585916625 4.741147613927922
[7.06872180e-05 1.40333006e-04 2.44922670e-04 3.96101478e-04
 3.12999597e-04 2.34581948e+05 6.94014674e+06 1.91341865e+02] 0.37941377585916625 4.741147613927922
[5.07711576e-05 1.58815526e-04 2.40359276e-04 2.62695931e-04
 5.83489143e-04 2.29465431e+05 3.34589687e+06 1.94859203e+02] 0.3168171530190623 3.778338084925309
[4.61993967e-05 2.13372980e-04 1.79239049e-04 4.25035129e-04
 4.21420849e-04 3.37447355e+05 3.04607738e+06 1.69364387e+02] 0.2919580451991701 -19.323664890529585
[5.33791880e-05 1.68867700e-04 2.52122394e-04 2.74816715e-04
 6.91259443e-04 2.55900961e+05 5.55586244e+06 1.76985584e+02] 0.35313740163752544 5.337083831733651
[8.62399382e-05 1.29718870e-04 2.69931229e-04 4.14550842e-04
 5.18567762e-04 3.15953359e+05 4.10290695e+06 1.76040410e+02] 0.31088905830463526 0.5034149018395908
[6.10701793e-05 1.50268527e-04 3.

In [18]:
result.x

array([0.50771158, 1.58815526, 2.40359276, 2.62695931, 5.83489143,
       2.29465431, 3.34589687, 1.94859203])